Algorithme de recommandation de films





Introduction:

Au cours des dernières décennies, avec la montée du nombre d’utilisateurs sur les réseaux sociaux et de nombreux autres services Web (Amazon, Netflix …), les systèmes de recommandation ont pris de plus en plus de place dans nos vies. Du e-commerce à la publicité en ligne , les systèmes de recommandation sont aujourd'hui incontournables dans nos parcours quotidiens en ligne.
D’une façon très générale, les systèmes de recommandation sont des algorithmes visant à proposer des éléments pertinents aux utilisateurs ( du texte à lire, des produits à acheter, des films à regarder, ou tout autre élément selon le secteur ). Notre projet est ainsi de construire un algorithme de recommandation à partir des préférences utilisateurs.  
L’objectif de ce projet est d’implémenter un système de recommandation en se basant sur des algorithmes de machine learning. Le but sera donc de chercher les films à recommander aux utilisateurs  en se basant sur différentes informations fournies par les utilisateurs concernant leurs préférences.


Méthodologie:

1-Collecte des données:

Les données utilisées pour cette analyse proviennent d’une plateforme web organisant des compétitions en science des données nommée Kaggle. Plus exactement,  notre data a ete telechargee a partir du la page suivante: ‘https://www.kaggle.com/rounakbanik/the-movies-dataset’. On dispose en tout de 7 tables de données films dont on va utiliser que 3: movies_metadat.csv, keywords,csv et ratings.csv.
Movies_metadata,csv contient des métadonnées pour les 45000 films répertoriés dans Full MovieLens. Ratings.csv regroupe 26 millions d'évaluations attribueses par 270 000 utilisateurs pour ces 45 000 films. Les notes sont sur une échelle de 1 à 5 et ont été obtenues sur le site officiel de GroupLens. Keywords.csv contient les mots cles pour chaque film.
La premiere etape connsiste a importer les packages necessaires pour la manipulation des donnees et d'importer nos dataframes: 

In [16]:
import pandas as pd
import numpy as np
import matplotlib as plt
import ast
from ast import literal_eval


#IMPORT DATA
metadata = pd.read_csv("metadata_carac_speciaux_new.csv")
keywords=pd.read_csv('keywords_carac_speciaux_new.csv', delimiter = ',')
ratings=pd.read_csv('/home/fitec/donnees_films/ratings.csv', delimiter = ',')



La commande .info() permet delister le noms des variables, leur types ainsi que la taille de la base de donnees: 

In [11]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45463 entries, 0 to 45462
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45463 non-null  bool   
 1   belongs_to_collection  45463 non-null  object 
 2   budget                 45463 non-null  float64
 3   genres                 45463 non-null  object 
 4   homepage               7779 non-null   object 
 5   id                     45463 non-null  int64  
 6   imdb_id                45446 non-null  object 
 7   original_language      45452 non-null  object 
 8   original_title         42611 non-null  object 
 9   overview               44504 non-null  object 
 10  popularity             45463 non-null  float64
 11  poster_path            45077 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45376 non-null  object 
 15  re

In [3]:
keywords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46419 entries, 0 to 46418
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        46419 non-null  int64 
 1   keywords  46419 non-null  object
dtypes: int64(1), object(1)
memory usage: 725.4+ KB


In [5]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26024289 entries, 0 to 26024288
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 794.2 MB


Après une etude approffondie des donnees et de nos besoins on a decide de ne garder que les variables (id', 'genres', 'original_language', 'production_companies','production_countries', 'release_date', 'title'  , 'vote_average', 'vote_count', 'adult_False', 'adult_True' ) dans la dataset Movies_metadata.csv.

2-Nettoyage:

3-Traitement:

Après nettoyage on dispose de nos données corrigées et sans caractères spéciaux. On commence donc l’étape préparation et traitement des variables. D'abord on supprime le valeurs manquantes (missing). Ensuite on selectionne que le films dejasorties en cinema avec la variable 'released'. Puis on transforme la varible adult en variable categorielle pour faciliter le traitement. Enfin on supprime toute repetition et redondance dans notre data.

In [17]:
#1 supprimer les valeurs missing
metadata=metadata.dropna(subset=['id'])
metadata=metadata.dropna(subset=['title'])
#2 selectionner les film=released
metadata=metadata.loc[metadata['status']== 'Released']
#3 encode adult var
metadata=pd.get_dummies(metadata, columns=["adult"])
#4 drop duplicates  
metadata=metadata.drop_duplicates()
metadata=metadata.drop_duplicates(subset='id', keep="first")

On selectionne que les variables pertinentes pour notre etude:

In [18]:
metadata=metadata[['genres', 
                     'id',
                     'original_language', 
                     'production_companies', 
                     'production_countries', 
                     'release_date',
                     'title'  , 
                     'vote_average',
                     'vote_count',
                     'adult_False', 
                     'adult_True'      ]]




Comme on peut le constater les variables qualitatives telles que genres, original_language, production_companies, production_coutries et keywords sont sous forme de dictionnaires et nécessitent donc un traitement a part. Avec la fonction 'categorie' on peut transformer les valeurs dictionnaires en string. Et avec la fonction 'get_dic' on transforme les valeurs en variables catégorielles pour enrichir d'avantage nos données.  


In [19]:
metadata['genres'].head(3) #brute

0    [{'id':16,'name':'Animation'},{'id':35,'name':...
1    [{'id':12,'name':'Adventure'},{'id':14,'name':...
2    [{'id':10749,'name':'Romance'},{'id':35,'name'...
Name: genres, dtype: object

In [20]:


###############################fonction pr obtenir une liste des categories
def categorie (data, variable):
    data['genrestest'] = data[variable].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
    liste=data['genrestest'].str[0].value_counts()[:5].index.tolist()
    return liste
#######################################################

liste_genre=categorie(metadata, 'genres')
liste_pcomp=categorie(metadata, 'production_companies')
liste_pcount=categorie(metadata, 'production_countries')

metadata=metadata.drop(['genrestest'], axis=1)

#########################fonction encoding pour attribuer 1 a chaque element qui existe dans la liste des categories############################
def encoding_dic(data, variable, liste):

    serie_col = data[variable]
    #Création de la colonne total : liste des catégories appartenant à la liste pour chaque ligne
    def add(x, liste_col):
        total = []
        if type(x) == str and x[0] == "[":
            a = ast.literal_eval(x)
            if len(a) > 0:
                for j in range(len(a)):
                    comp = a[j]["name"]
                    if comp in liste_col:
                        total.append(comp)
                if len(total) == 0:
                    total.append("null")
            else:
                total.append("null")
        return total
    
    total = serie_col.apply(lambda x : add(x, liste_col = liste))
    df = serie_col.to_frame()
    df["total"] = total
    
    #Création des colonnes pour le OneHotEncoding
    for genre in liste:
        df[genre] = 0
    
    #Complétion des colonnes OneHotEncoding grâce à la colonne total
    def add2(x,genre_cherche):
        for genre in x["total"]:
            if genre == genre_cherche:
                return 1
        return 0
    
    for genre in liste:
        df[genre] = df.apply(lambda x : add2(x, genre_cherche = genre), axis=1)
    
    return df



On selectionne les ctegories les plus utilisees dans chaque variable:

In [21]:
liste_genre = ['Drama', 'Comedy', 'Thriller', 'Romance', 'Action', 'Horror', 'Crime', 'Documentary']
liste_prod_comp = ['WarnerBros.', 'Metro-Goldwyn-MayerMGM', 'ParamountPictures', 'TwentiethCenturyFoxFilmCorporation', 'UniversalPictures', 'ColumbiaPicturesCorporation', 'Canal', 'ColumbiaPictures', 'RKORadioPictures']
liste_prod_count = ['UnitedStatesofAmerica', 'null', 'UnitedKingdom', 'France', 'Germany', 'Italy', 'Canada', 'Japan', 'Spain', 'Russia']


On fait un drop des variables qu'on en a plus besoin apres transformation et on fais la concatenation en ajoutant les nouvelles categories a notre dataframe originale:

In [22]:
dfgenres = encoding_dic(data=metadata, variable="genres", liste=liste_genre)
dfgenres=dfgenres.drop(['genres', 'total'], axis=1)
dfprodcomp = encoding_dic(data=metadata, variable="production_companies", liste=['null', 'Warner Bros.', 'Metro-Goldwyn-Mayer (MGM)', 'Paramount Pictures'])
dfprodcomp=dfprodcomp.drop(['production_companies', 'total'], axis=1)
dfprodcount = encoding_dic(data=metadata, variable="production_countries", liste=liste_pcount)
dfprodcount=dfprodcount.drop(['production_countries', 'total'], axis=1)

datamovienew=pd.concat([metadata, dfgenres, dfprodcomp, dfprodcount], axis=1)

datamovienew=datamovienew.drop(['genres', 'production_companies', 'production_countries'], axis=1)


In [25]:
datamovienew.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44955 entries, 0 to 45462
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         44955 non-null  int64  
 1   original_language          44945 non-null  object 
 2   release_date               44878 non-null  object 
 3   title                      44955 non-null  object 
 4   vote_average               44955 non-null  float64
 5   vote_count                 44955 non-null  int64  
 6   adult_False                44955 non-null  uint8  
 7   adult_True                 44955 non-null  uint8  
 8   Drama                      44955 non-null  int64  
 9   Comedy                     44955 non-null  int64  
 10  Thriller                   44955 non-null  int64  
 11  Romance                    44955 non-null  int64  
 12  Action                     44955 non-null  int64  
 13  Horror                     44955 non-null  int

Les memes etapes pour le traitement de la dataframe keywords et on fait une jointure avec datamovienew pour recuperer les mots cles associes a chaque film:

In [26]:
keywords=keywords.dropna(subset=['id'])
keywords=keywords.drop_duplicates(subset='id', keep="first")

liste_key=['woman director', 'independent film', 'murder', 'based on novel', 'musical', 'sex', 'violence', 'nudity', 'biography', 'revenge', 'suspense', 'love', 'female nudity', 'sport', 'police', 'teenager', 'duringcreditsstinger', 'sequel', 'friendship', 'world war ii', 'drug', 'prison', 'stand-up comedy', 'high school', 'martial arts', 'suicide', 'kidnapping', 'rape', 'silent film', 'film noir', 'family', 'serial killer', 'monster', 'alien', 'dystopia', 'paris', 'new york', 'blood', 'gay', 'short', 'marriage', 'christmas', 'gore', 'zombie', 'death', 'gangster', 'small town', 'london england', 'romance', 'prostitute', 'detective', 'aftercreditsstinger', 'male nudity', 'robbery', 'vampire', 'father son relationship', 'wedding', 'los angeles', 'escape', 'dog', 'teacher', 'holiday', 'war', 'magic', 'hospital', 'doctor', 'music', 'remake', 'jealousy', 'based on true story', 'ghost', 'party', 'island', 'spy', 'new york city', 'lgbt', 'japan', 'daughter', 'investigation', 'coming of age', 'money', 'superhero', 'infidelity', 'corruption', 'torture', 'brother brother relationship', 'homosexuality', 'nazis', 'adultery', 'extramarital affair', 'wife husband relationship', 'slasher', 'supernatural', 'lawyer', 'dark comedy', 'friends', 'scientist']
dfkey= encoding_dic(data=keywords, variable="keywords", liste=liste_key)
dfkey=dfkey.drop(['keywords', 'total'], axis=1)

datakey=pd.concat([keywords, dfkey], axis=1)
datakey=datakey.drop(['keywords'], axis=1)

datamovienew=pd.merge(datamovienew,datakey, on='id')




On transforme la variable type date 'release_date' en une variable qualitative avec les categories suivantes:date inconnue,
films anciens, films récents et films très récents pour regrouper les films ayant une date de sorties dans un meme intervale du temps. Avec la commande get_dummies on aura directement les variables categorielles correspondantes:

In [ ]:
#8 Catégorisation de la variable release_date
var = []
a0 = "date inconnue"
a1 = "films anciens"
a2 = "films récents"
a3 = "films très récents"

dates = datamovienew["release_date"]
a = dates.apply(lambda x : str(x))
a = pd.DataFrame(a.apply(lambda x : x[0:4]))
francaise, italienne
for i in range(0,len(a)):
    if len(a.loc[i,'release_date']) < 4:
        var.append(a0)
    elif (len(a.loc[i,'release_date']) >= 4 and int(a.loc[i,'release_date']) <= 1990) :
        var.append(a1)
    elif (len(a.loc[i,'release_date']) >= 4 and 1990 < int(a.loc[i,'release_date']) <= 2010) :
        var.append(a2)
    elif (len(a.loc[i,'release_date']) >= 4 and int(a.loc[i,'release_date']) > 2010):
        var.append(a3)
    else:
        var.append(a0)

datamovienew["dates_types"] = var
datamovienew=pd.get_dummies(datamovienew, columns=["dates_types"])
datamovienew=datamovienew.drop(['release_date'], axis=1)





Finalement pour la variable originale_language on garde que la langue anglaise, italienne, japonaise et allemande Egalement ondefinit les categories avec get_dummies:

In [27]:
#9 Catégorisation de la variable original language
datamovienew["original_language"].unique()

def only_these_languages(x):
    if x not in ["fr", "en", "it", "ja", "de"]:
        return "other"
    else:
        return x
    
datamovienew["original_language"] = datamovienew["original_language"].apply(lambda x : only_these_languages(x))  
datamovienew=pd.get_dummies(datamovienew, columns=["original_language"])
final_data_movie = datamovienew


voici la dataframe finale qui sera objet de notre analyse machine learning:

In [28]:
final_data_movie

,id,release_date,title,vote_average,vote_count,adult_False,adult_True,Drama,Comedy,Thriller,...,lawyer,dark comedy,friends,scientist,original_language_de,original_language_en,original_language_fr,original_language_it,original_language_ja,original_language_other
0,862,1995-10-30,ToyStory,7.7,5415,1,0,0,1,0,...,0,0,1,0,0,1,0,0,0,0
1,8844,1995-12-15,Jumanji,6.9,2413,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,15602,1995-12-22,GrumpierOldMen,6.5,92,1,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,31357,1995-12-22,WaitingtoExhale,6.1,34,1,0,1,1,0,...,0,0,0,0,0,1,0,0,0,0
4,11862,1995-02-10,FatheroftheBridePartII,5.7,173,1,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44949,439050,NaN,Subdue,4.0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
44950,111109,2011-11-17,CenturyofBirthing,9.0,3,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
44951,67758,2003-08-01,Betrayal,3.8,6,1,0,1,0,1,...,0,0,0,0,0,1,0,0,0,0
44952,227506,1917-10-21,SatanTriumphant,0.0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


4-Machine learning:
    
    

In [29]:
import os
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.preprocessing import StandardScaler
import datetime

In [30]:

################### Parametres simulation ##############################
remove_col_kmeans_movies = ['id','title','vote_average', 'vote_count']
trunc_user_high = 400 #nombre max de vues total par user
trunc_user_low = 20 #nombre min de vues total par user
trunc_movie_low = 1000
trunc_movie_high = 100000
coude_centroid_movies = 20
kmeans_centroid_movies = 4
coude_centroid_users  =9
kmeans_centroid_users = 4
##########################################################################

On a deja importe la table ratings on supprime la variable 'timestamp' et on garde que les ids des films qui sont aussi dans la precedante table final_data_movie:

In [31]:

tableau_movies_full = final_data_movie
#ratings = pd.read_csv(input_dir + "ratings.csv")
ratings  = ratings.drop(['timestamp'], axis = 1)

all_movies = list(tableau_movies_full.drop_duplicates("id")["id"])
ratings = ratings[ratings["movieId"].isin(all_movies)]

nbr_votes_movie est un dataframe qui regroupe la somme des films selon leur id:

In [33]:
nbr_votes_movie = ratings.groupby("movieId")["movieId"].count().reset_index(name= "count_movie")



In [34]:
nbr_votes_movie

,movieId,count_movie
0,2,26060
1,3,15497
2,5,15258
3,6,27895
4,11,19475
...,...,...
7497,176077,1
7498,176085,2
7499,176143,1
7500,176167,1


On fait une jointure entre la table ratings et la table nbr_votes_movie. On peut ainsi effectur des conditions selon le nombre de films notes.
Chaque film doit etre note au moins 'trunc_movie_low' fois et  ne doit pas etre noter plus que 'trunc_movie_high' fois pour etre selectionner.
Ceci est dans le but d'obtenir des resultat plus pertinents en illiminant les cas extremes: le films tres populaires notes par des milliers d'utilisateurs et vice versa.
Dans ce cas trunc_movie_low est egal a 1000 et trunc_movie_high est egal a 100000.

In [35]:

ratings = pd.merge(ratings, nbr_votes_movie, left_on="movieId", right_on='movieId', how='inner')
ratings = ratings[ratings["count_movie"]>trunc_movie_low]
ratings = ratings[ratings["count_movie"]<trunc_movie_high]
ratings = ratings.drop("count_movie", axis= 1)

#legit_movies = list(ratings.drop_duplicates("movieId")["movieId"])
#tableau_movies = tableau_movies_full[tableau_movies_full["id"].isin(legit_movies)]


On supprime les vriables dont on en a plus besoin:

In [37]:
del nbr_votes_movie
#del legit_movies
del all_movies

tableau_movies = tableau_movies_full.drop(tableau_movies_full[remove_col_kmeans_movies], axis = 1)


De la meme maniere que pour les films, on selectionne aussi les utilisateurs qui n'ont pas trop (ni pas assez ) note les films.
Les deux prametres correspondant aux utilisateurs sont trunc_user_low (=20) et trunc_user_low (=400). Apres selection on aura la dataframe finale ratings qu'on utilisera pour le k-means utilisateurs.

In [38]:
data_user_votes = ratings.groupby(["userId"])["rating"].apply(lambda x : len(list(x) )).reset_index(name = 'voteCount')
data_user_votes = data_user_votes[ trunc_user_low  < data_user_votes['voteCount'] ]
data_user_votes = data_user_votes[  data_user_votes['voteCount'] < trunc_user_high]
df = data_user_votes.sort_values(by=['voteCount'])

ratings = ratings[np.isin(ratings['userId'], data_user_votes['userId'])]


In [40]:
ratings.head()

,userId,movieId,rating
1,11,110,3.5
2,22,110,5.0
3,24,110,5.0
4,29,110,3.0
5,30,110,5.0
